# Lesson2 表格型方法——Sarsa
## 1. Sarsa 简介
* `Sarsa`全称是`state-action-reward-state'-action'`，目的是学习特定的`state`下，特定`action`的价值`Q`，最终建立和优化一个`Q`表格，以`state`为行，`action`为列，根据与环境交互得到的`reward`来更新`Q`表格，更新公式为：

![](https://ai-studio-static-online.cdn.bcebos.com/776b473b7f994702a3e05c5eac1156a7ce03b9e6bdb5453085fa9cbb86979715)
* `Sarsa`在训练中为了更好的探索环境，采用`ε-greedy`方式来训练，有一定概率随机选择动作输出。

## 2. Sarsa 实战
* 使用`Sarsa`解决悬崖问题，找到绕过悬崖通往终点的路径。

### Step1 安装依赖

In [1]:
!pip install gym

ERROR: parl 1.3.1 requires flask>=1.0.4, which is not installed.
ERROR: parl 1.3.1 requires flask-cors, which is not installed.
ERROR: parl 1.3.1 requires pyarrow==0.13.0, which is not installed.
ERROR: parl 1.3.1 requires tb-nightly==1.15.0a20190801, which is not installed.
ERROR: parl 1.3.1 requires tensorboardX==1.8, which is not installed.
ERROR: parl 1.3.1 has requirement cloudpickle==1.2.1, but you'll have cloudpickle 1.3.0 which is incompatible.
ERROR: parl 1.3.1 has requirement pyzmq==18.0.1, but you'll have pyzmq 19.0.1 which is incompatible.


### Step2 导入依赖

In [2]:
import gym
import numpy as np
import time

### Step3 Agent
* `Agent`是和环境`environment`交互的主体。
* `predict()`方法：输入观察值`observation`（或者说状态`state`），输出动作值
* `sample()`方法：再`predict()`方法基础上使用`ε-greedy`增加探索
* `learn()`方法：输入训练数据，完成一轮`Q`表格的更新

In [3]:
# agent.py 
class SarsaAgent(object):
    def __init__(self, obs_n, act_n, learning_rate=0.01, gamma=0.9, e_greed=0.1):
        self.act_n = act_n      # 动作维度，有几个动作可选
        self.lr = learning_rate # 学习率
        self.gamma = gamma      # reward的衰减率
        self.epsilon = e_greed  # 按一定概率随机选动作
        self.Q = np.zeros((obs_n, act_n))

    # 根据输入观察值，采样输出的动作值，带探索
    def sample(self, obs):
        if np.random.uniform(0, 1) < (1.0 - self.epsilon): #根据table的Q值选动作
            action = self.predict(obs)
        else:
            action = np.random.choice(self.act_n) #有一定概率随机探索选取一个动作
        return action

    # 根据输入观察值，预测输出的动作值
    def predict(self, obs):
        Q_list = self.Q[obs, :]
        maxQ = np.max(Q_list)
        action_list = np.where(Q_list == maxQ)[0]  # maxQ可能对应多个action
        action = np.random.choice(action_list)
        return action

    # 学习方法，也就是更新Q-table的方法
    def learn(self, obs, action, reward, next_obs, next_action, done):
        """ on-policy
            obs: 交互前的obs, s_t
            action: 本次交互选择的action, a_t
            reward: 本次动作获得的奖励r
            next_obs: 本次交互后的obs, s_t+1
            next_action: 根据当前Q表格, 针对next_obs会选择的动作, a_t+1
            done: episode是否结束
        """
        predict_Q = self.Q[obs, action]
        if done:
            target_Q = reward # 没有下一个状态了
        else:
            target_Q = reward + self.gamma * self.Q[next_obs, next_action] # Sarsa
        self.Q[obs, action] += self.lr * (target_Q - predict_Q) # 修正q

    # 保存Q表格数据到文件
    def save(self):
        npy_file = './q_table.npy'
        np.save(npy_file, self.Q)
        print(npy_file + ' saved.')
    
    # 从文件中读取Q值到Q表格中
    def restore(self, npy_file='./q_table.npy'):
        self.Q = np.load(npy_file)
        print(npy_file + ' loaded.')


### Step4 Training && Test（训练&&测试）
* `run_episode()`：`agent`在一个`episode`中训练的过程，使用`agent.sample()`与环境交互，使用`agent.learn()`训练`Q`表格。
* `test_episode()`：`agent`在一个`episode`中测试效果，评估目前的`agent`能在一个`episode`中拿到多少总`reward`。

In [4]:
def run_episode(env, agent, render=False):
    total_steps = 0 # 记录每个episode走了多少step
    total_reward = 0

    obs = env.reset() # 重置环境, 重新开一局（即开始新的一个episode）
    action = agent.sample(obs) # 根据算法选择一个动作

    while True:
        next_obs, reward, done, _ = env.step(action) # 与环境进行一个交互
        next_action = agent.sample(next_obs) # 根据算法选择一个动作
        # 训练 Sarsa 算法
        agent.learn(obs, action, reward, next_obs, next_action, done)

        action = next_action
        obs = next_obs  # 存储上一个观察值
        total_reward += reward
        total_steps += 1 # 计算step数
        if render:
            env.render() #渲染新的一帧图形
        if done:
            break
    return total_reward, total_steps


def test_episode(env, agent):
    total_reward = 0
    obs = env.reset()
    while True:
        action = agent.predict(obs) # greedy
        next_obs, reward, done, _ = env.step(action)
        total_reward += reward
        obs = next_obs
        # time.sleep(0.5)
        # env.render()
        if done:
            break
    return total_reward

### Step5 创建环境和Agent，启动训练

In [5]:
# 使用gym创建悬崖环境
env = gym.make("CliffWalking-v0")  # 0 up, 1 right, 2 down, 3 left

# 创建一个agent实例，输入超参数
agent = SarsaAgent(
        obs_n=env.observation_space.n,
        act_n=env.action_space.n,
        learning_rate=0.1,
        gamma=0.9,
        e_greed=0.1)


# 训练500个episode，打印每个episode的分数
for episode in range(500):
    ep_reward, ep_steps = run_episode(env, agent, False)
    print('Episode %s: steps = %s , reward = %.1f' % (episode, ep_steps, ep_reward))

# 全部训练结束，查看算法效果
test_reward = test_episode(env, agent)
print('test reward = %.1f' % (test_reward))

145.0
Episode 12: steps = 171 , reward = -171.0
Episode 13: steps = 29 , reward = -29.0
Episode 14: steps = 101 , reward = -101.0
Episode 15: steps = 86 , reward = -185.0
Episode 16: steps = 135 , reward = -234.0
Episode 17: steps = 159 , reward = -357.0
Episode 18: steps = 98 , reward = -98.0
Episode 19: steps = 91 , reward = -91.0
Episode 20: steps = 99 , reward = -99.0
Episode 21: steps = 97 , reward = -97.0
Episode 22: steps = 90 , reward = -189.0
Episode 23: steps = 172 , reward = -469.0
Episode 24: steps = 63 , reward = -63.0
Episode 25: steps = 95 , reward = -194.0
Episode 26: steps = 65 , reward = -65.0
Episode 27: steps = 146 , reward = -344.0
Episode 28: steps = 92 , reward = -92.0
Episode 29: steps = 108 , reward = -207.0
Episode 30: steps = 29 , reward = -29.0
Episode 31: steps = 79 , reward = -178.0
Episode 32: steps = 85 , reward = -184.0
Episode 33: steps = 83 , reward = -83.0
Episode 34: steps = 49 , reward = -49.0
Episode 35: steps = 86 , reward = -86.0
Episode 36: ste